# Facial Keypoint Detection


## Initialize Environment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
from tensorflow.keras.layers import LeakyReLU, ReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Convolution2D, BatchNormalization, Flatten
from tensorflow.keras.layers import Dense, Dropout, Input, MaxPool2D
from tensorflow.keras.optimizers import Nadam
from keras import backend
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import cv2
import os, gc, json, math

Using TensorFlow backend.
/home/user1/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user1/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user1/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user1/anaconda3/lib/python3.7/site-packages/tensorflo

In [2]:
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)

2.2.4
1.13.1


## Functions

In [3]:
def convert_pixels(data):
    """
    Convert pixels to the right intensity 0-1 and in a square matrix.
    """
    data = np.array([row.split(' ') for row in data['Image']],dtype='float') / 255.0
    data = data.reshape(-1,96,96,1)
    return(data)

In [4]:
def view_img(sample_img,coord=None):
    """
    Display an image. For debugging, display a coordinate on the image.
    input:
        - sample_img: numpy array. image to be displayed
        - coord: lst. of coordinates in form [[x_coordinate,y_coordinate],[x_coordinate,y_coordinate]]
    TODO handle multiple coordinates. Work out bugs with multiple coordinates
    """
    plt.figure()
    plt.imshow(sample_img.reshape(96,96),cmap='gray')
    if coord is not None:
        plt.scatter(coord[0],coord[1],marker = '*',c='r')
    plt.show()

In [5]:
def get_facial_keypoints(data,ind):
    """
    Structure the coordinates for all facial keypoints for a single image.
    inputs:
        - data: numpy array containing rows as each image sample and columns as facial keypoint coordinates
        - ind: index of the image
    output:
        - numpy array with format [[list of x-coordinates],[list of y-coordinates]]
    """
    data[ind]
    it = iter(data[ind])
    x_coord = []
    y_coord = []

    for x in it:
        x_coord.append(x)
        y_coord.append(next(it))
    
    return(np.array([x_coord,y_coord]))


## Import Data

In [6]:
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
reset_keras()    

20


In [7]:
if not os.path.exists('../output/'):
    os.makedirs('../output/model')
    os.makedirs('../output/history')
    
    
model_dir = "../output/model/"
history_dir = "../output/history/"

train_file = '../input/training/training.csv'
test_file = '../input/test/test.csv'
train_data = pd.read_csv(train_file)  
#test_data = pd.read_csv(test_file)


bad_samples = [1747, 1731, 1877, 1881, 1979, 2199, 2289, 2321, 2453, 3173, 3296, 3447, 4180, 6859,
              2090, 2175, 1907, 2562, 2818, 3296, 3447, 4263, 4482, 4490, 4636, 5059, 6493, 6585, 6906]

train_data = train_data.drop(bad_samples).reset_index(drop=True)
train_clean = train_data.dropna()


Create training vector with images and normalize thee

In [8]:
x_train = convert_pixels(train_data)
x_clean = convert_pixels(train_clean)
y_train = train_data[[col for col in train_data.columns if col != 'Image']].to_numpy()
y_clean = train_clean[[col for col in train_clean.columns if col != 'Image']].to_numpy()

Generate labels 

In [9]:

#imputer = KNNImputer(n_neighbors=3, weights='distance')

imputer = IterativeImputer(max_iter=1000, tol=0.01, random_state=42)
y_train = imputer.fit_transform(y_train)



bad_bottom_lip = [210, 350, 499, 512, 810, 839, 895, 1058, 1194,1230, 1245, 1546, 1548]

def adjust_mouth_coord(y_train):
    

    for sample in bad_bottom_lip:
        y_train[sample][29] = 94
        y_train[sample][28] = y_train[sample][26]
          
    
    
    for sample in range(len(y_train)):
        if(y_train[sample][29] < y_train[sample][27]+1):
             y_train[sample][27] = y_train[sample][29] -1
   
        if((y_train[sample][23] + y_train[sample][25]) > (2*y_train[sample][29]-1)):
             diff = y_train[sample][23] - y_train[sample][25]
       
             if(diff > 0):
                y_train[sample][23] = y_train[sample][29] -1
                y_train[sample][25] = y_train[sample][29] -1 - diff
           
             else:
                y_train[sample][23] = y_train[sample][29] -1 + diff
                y_train[sample][25] = y_train[sample][29] -1 
                
    return(y_train)           

y_train = adjust_mouth_coord(y_train)    
 
    
   

## Feature Engineering

### Set feature engineering parameters

In [10]:
fill_na = False
add_flip_horiz = True
add_blur_img = False
add_rotate_img = False
add_contrast_img = True
add_translate_img = False
orig_x_train = x_train.copy()
orig_y_train = y_train.copy()


### Fill NA in the training labels.

In [ ]:
if fill_na:
    # https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns
    # get column means
    col_mean = np.nanmean(y_train,axis=0)

    # find the x,y indices that are missing from y_train
    inds = np.where(np.isnan(y_train))

    # fill in missing values in y_train with the column means. "take" is much more efficient than fancy indexing
    y_train[inds] = np.take(col_mean, inds[1])


### Flip images horizontally and add to the training data

In [11]:
def flip_img_horiz(train_data):
    """
    Flip images horizontally for all training images
    """
    # Flip images
    x_train = convert_pixels(train_data)
    flip_img = np.array([np.fliplr(x_train[[ind]][0]) for ind in range(x_train.shape[0])])
    
    # Flip coordinates
    train_data_flip = train_data.copy()
    x_columns = [col for col in train_data.columns if '_x' in col]
    train_data_flip[x_columns] = train_data[x_columns].applymap(lambda x: 96-x)
    
    #left and right are swapped so undo
    left_columns = [col for col in train_data.columns if 'left' in col]
    right_columns = [col for col in train_data.columns if 'right' in col]
    train_data_flip[left_columns+right_columns] = train_data_flip[right_columns+left_columns]
    
    flip_coord = train_data_flip[[col for col in train_data if col != 'Image']].to_numpy()
    return(flip_img,flip_coord)

if add_flip_horiz:
    # Apply the augmentation and add the new data to the training set
    flipped_img,flipped_coord = flip_img_horiz(train_data)
    
    imputer = IterativeImputer(max_iter=1000, tol=0.01, random_state=3)
    flipped_coord = imputer.fit_transform(flipped_coord)
    flipped_coord = adjust_mouth_coord(flipped_coord)
    x_train = np.append(x_train,flipped_img,axis=0)
    y_train = np.append(y_train,flipped_coord,axis=0)
    
    
    

### Add Gaussian blurring with a 5x5 filter with $\sigma$ = 2.

In [ ]:
def blur_img():
    """
    Add Gaussian blurring to the images
    """
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
    blur_img = np.array([cv2.GaussianBlur(orig_x_train[[ind]][0],(5,5),2).reshape(96,96,1) for ind in range(orig_x_train.shape[0])])
    
    return(blur_img)

if add_blur_img:
    x_train = np.append(x_train,blur_img(),axis=0)
    y_train = np.append(y_train,orig_y_train,axis=0)

### Add image rotation

In [ ]:
def rotate_img(x_train, y_train):
    """"
    Rotate images by angles between [5, 10, 14 degrees]
    """
    angles = [5, -5, 10, -10, 14, -14]
    b = np.ones((1,3))
    rows,cols = (96,96)
    x_train_rot = []
    y_train_rot = y_train.copy()
    M_angles = [cv2.getRotationMatrix2D((cols/2,rows/2),angle,1) for angle in angles]
    
    for i in range(x_train.shape[0]):
        #M = cv2.getRotationMatrix2D((cols/2,rows/2),np.random.choice(angles,1),1)
        M = M_angles[np.random.choice(len(M_angles))]
        x_train_rot.append((cv2.warpAffine(x_train[[i]].reshape(rows,cols,1),M,(cols,rows)).reshape(96,96,1)))
       
        #apply affine transformation to (x,y) labels
        for j in range(int(y_train.shape[1]/2)):
            b[:,0:2] = y_train[i,2*j:2*j+2]
            y_train_rot[i,2*j:2*j+2] = np.dot(b,M.transpose()) 
    
    x_train_rot = np.array(x_train_rot)
    return x_train_rot, y_train_rot

if add_rotate_img:
    
    x_rotate, y_rotate = rotate_img(x_train,y_train)
    x_train = np.append(x_train,x_rotate,axis=0)
    y_train = np.append(y_train,y_rotate,axis=0)   
    

### Add image contrast

In [12]:
def contrast_img(apply_random,brightness=.5):
    """
    Add brighter and darker images to the training set with the range of pixel values allowed set at 0 and 1
    Only applies contrast to the original images. Need to make sure we're not setting random seed
    input:
        - brightness: float between -1 and 1
        - apply_random: boolean. Applies random brightness to every sample in the data set.
          Ignores brightness setting
    """
    if apply_random:
        # uses a .1 increment [-.3,.8) to pick out a brightness number for each sample
        # numbers are chosen such that they are still realistically visible and
        # the added training data has an appreciable change in contrast
        brightness = np.random.choice(np.round(np.arange(-.3,.8,.1),2),size=orig_x_train.shape[0])
        bright_img = np.array([orig_x_train[[ind]][0]+brightness[ind] for ind in range(orig_x_train.shape[0])])
    else:
        bright_img = orig_x_train + brightness

    bright_img[bright_img > 1] = 1
    bright_img[bright_img < 0] = 0
    return(bright_img)
    
if add_contrast_img:
    # create two sets of images undergoing contrast changes
    x_train = np.append(x_train,contrast_img(apply_random=True),axis=0)
    y_train = np.append(y_train,orig_y_train,axis=0)
    
# testing code
# view_img(contrast_img(apply_random=True)[[30]])
# view_img(contrast_img(apply_random=True)[[30]])
# view_img(contrast_img(apply_random=True)[[30]])


In [ ]:
def translate_img():
    """
    Add translational shift to the images randomly
    """
    trans_train_data = train_data.copy()
    
    rows,cols = (96,96)
    shift_x = 96*np.random.choice(np.arange(-.4,.4,.05),size=trans_train_data.shape[0])
    shift_y = 96*np.random.choice(np.arange(-.4,.4,.05),size=trans_train_data.shape[0])
    M = [np.float32([[1,0,shift_x[ind]],[0,1,shift_y[ind]]]) for ind in range(trans_train_data.shape[0])]
    trans_img = np.array([cv2.warpAffine(x_train[[ind]].reshape(96,96,1),M[ind],(cols,rows)).reshape(96,96,1) for ind in range(trans_train_data.shape[0])])
    
    x_col = [col for col in train_data.columns if ((col != 'Image') & ('_x' in col))]
    y_col = [col for col in train_data.columns if ((col != 'Image') & ('_y' in col))]

    shift_x_array = np.array([np.repeat(x,len(x_col))for x in shift_x])
    shift_y_array = np.array([np.repeat(y,len(y_col))for y in shift_y])
    
    trans_train_data[x_col] += shift_x_array
    trans_train_data[y_col] += shift_y_array
    
    trans_coord = np.array(trans_train_data[[col for col in trans_train_data.columns if col != 'Image']])

    # TODO should we force these to be nan or leave as is? If leave as is, comment this out.
    trans_coord[trans_coord > 96]= np.nan
    trans_coord[trans_coord < 0]= np.nan
    return(trans_img,trans_coord)

if add_translate_img:
    trans_img,trans_coord = translate_img()
    x_train = np.append(x_train,trans_img,axis=0)
    y_train = np.append(y_train,trans_coord,axis=0)


## Modeling

In [13]:
# Define callback function if detailed log required
class History(tensorflow.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_loss = []
        self.train_rmse = []
        self.val_rmse = []
        self.val_loss = []

    def on_batch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.train_rmse.append(logs.get('rmse'))
        
    def on_epoch_end(self, batch, logs={}):    
        self.val_rmse.append(logs.get('val_rmse'))
        self.val_loss.append(logs.get('val_loss'))
        
# Implement ModelCheckPoint callback function to save CNN model
class CNN_ModelCheckpoint(tensorflow.keras.callbacks.Callback):

    def __init__(self, model, filename):
        self.filename = filename
        self.cnn_model = model

    def on_train_begin(self, logs={}):
        self.max_val_rmse = math.inf
        
 
    def on_epoch_end(self, batch, logs={}):    
        val_rmse = logs.get('val_rmse')
        if(val_rmse < self.max_val_rmse):
           self.max_val_rmse = val_rmse
           self.cnn_model.save_weights(self.filename)


In [14]:
def final_model():
    model_input = Input(shape=(96,96,1))

    x = Convolution2D(32, (3,3), padding='same', use_bias=False)(model_input)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(32, (3,3), padding='same', use_bias=False)(model_input)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Convolution2D(64, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(64, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Convolution2D(96, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(96, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Convolution2D(128, (3,3),padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(128, (3,3),padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    #x = Dropout(0.1)(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Convolution2D(256, (3,3),padding='same',use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(256, (3,3),padding='same',use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Convolution2D(512, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(512, (3,3), activation='relu', padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(512,activation='relu')(x)
    x = Dropout(0.1)(x)
    model_output = Dense(30)(x)
    model = Model(model_input, model_output, name="final_model")
    return model

model = final_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        288       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)    

In [15]:
# Custom RMSE metric
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

#from tensorflow.keras.optimizers.schedules import InverseTimeDecay

# Use Nadam optimizer with variable learning rate
optimizer = Nadam(lr=0.00001,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)


# Loss: MSE and Metric = RMSE
model.compile(optimizer= optimizer, 
              loss='mean_squared_error',
              metrics=[rmse])

#Callback to save the best model
saveBase_Model = CNN_ModelCheckpoint(model, model_dir+"final_model_weights.h5")

#define callback functions
callbacks = [#EarlyStopping(monitor='val_rmse', patience=3, verbose=2),
             saveBase_Model]

Instructions for updating:
Use tf.cast instead.


Run for 1000 epochs and keeping 20% train-valid split

In [23]:

USE_SAVED_MODEL = False

if USE_SAVED_MODEL == False:
    history = model.fit(x_train,
                    y_train,
                    epochs = 1300,
                    batch_size = 256,
                    validation_split = 0.2, #data = (x_test, y_test),
                    callbacks = callbacks
                    )
    plt.style.use('seaborn-darkgrid')
    plt.plot(history.history['rmse'])
    plt.xlabel('Epochs')
    plt.ylabel('Root Mean Square Error')
    plt.title('Model Training Error')
    plt.show() 
    
else:
    model.load_weights(model_dir+"final_model_weights.h5")
                    

Train on 16852 samples, validate on 4214 samples
Epoch 1/1300
16852/16852 [==============================] - 14s 820us/sample - loss: 2.1682 - rmse: 1.3865 - val_loss: 1.9043 - val_rmse: 1.1371
Epoch 2/1300
16852/16852 [==============================] - 12s 724us/sample - loss: 2.1574 - rmse: 1.3824 - val_loss: 1.9065 - val_rmse: 1.1419
Epoch 3/1300
16852/16852 [==============================] - 12s 711us/sample - loss: 2.1841 - rmse: 1.3908 - val_loss: 1.9222 - val_rmse: 1.1447
Epoch 4/1300
16852/16852 [==============================] - 12s 710us/sample - loss: 2.1612 - rmse: 1.3836 - val_loss: 1.9844 - val_rmse: 1.1678
Epoch 5/1300
16852/16852 [==============================] - 12s 710us/sample - loss: 2.1343 - rmse: 1.3756 - val_loss: 1.9544 - val_rmse: 1.1549
Epoch 6/1300
16852/16852 [==============================] - 12s 714us/sample - loss: 2.1573 - rmse: 1.3825 - val_loss: 1.9271 - val_rmse: 1.1460
Epoch 7/1300
16852/16852 [==============================] - 12s 712us/sample - lo

Epoch 57/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 2.0946 - rmse: 1.3593 - val_loss: 1.9570 - val_rmse: 1.1443
Epoch 58/1300
16852/16852 [==============================] - 12s 694us/sample - loss: 2.1009 - rmse: 1.3610 - val_loss: 1.9180 - val_rmse: 1.1328
Epoch 59/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 2.0752 - rmse: 1.3547 - val_loss: 1.8891 - val_rmse: 1.1238
Epoch 60/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 2.0871 - rmse: 1.3574 - val_loss: 1.8830 - val_rmse: 1.1303
Epoch 61/1300
16852/16852 [==============================] - 12s 707us/sample - loss: 2.0761 - rmse: 1.3535 - val_loss: 1.8537 - val_rmse: 1.1178
Epoch 62/1300
16852/16852 [==============================] - 12s 725us/sample - loss: 2.0790 - rmse: 1.3530 - val_loss: 1.8666 - val_rmse: 1.1207
Epoch 63/1300
16852/16852 [==============================] - 12s 742us/sample - loss: 2.0935 - rmse: 1.3596 - val_loss: 1.97

16852/16852 [==============================] - 12s 695us/sample - loss: 2.0174 - rmse: 1.3333 - val_loss: 1.8690 - val_rmse: 1.1102
Epoch 114/1300
16852/16852 [==============================] - 12s 693us/sample - loss: 2.0078 - rmse: 1.3305 - val_loss: 1.8372 - val_rmse: 1.1019
Epoch 115/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 1.9977 - rmse: 1.3276 - val_loss: 1.8385 - val_rmse: 1.1012
Epoch 116/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 2.0284 - rmse: 1.3366 - val_loss: 1.8724 - val_rmse: 1.1119
Epoch 117/1300
16852/16852 [==============================] - 12s 698us/sample - loss: 2.0106 - rmse: 1.3312 - val_loss: 1.8752 - val_rmse: 1.1168
Epoch 118/1300
16852/16852 [==============================] - 12s 693us/sample - loss: 2.0180 - rmse: 1.3325 - val_loss: 1.8880 - val_rmse: 1.1141
Epoch 119/1300
16852/16852 [==============================] - 12s 693us/sample - loss: 1.9997 - rmse: 1.3273 - val_loss: 1.8864 - val

Epoch 169/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 1.9478 - rmse: 1.3072 - val_loss: 1.8600 - val_rmse: 1.1048
Epoch 170/1300
16852/16852 [==============================] - 12s 699us/sample - loss: 1.9617 - rmse: 1.3118 - val_loss: 1.7791 - val_rmse: 1.0810
Epoch 171/1300
16852/16852 [==============================] - 12s 707us/sample - loss: 1.9576 - rmse: 1.3112 - val_loss: 1.8434 - val_rmse: 1.0940
Epoch 172/1300
16852/16852 [==============================] - 12s 694us/sample - loss: 1.9676 - rmse: 1.3125 - val_loss: 1.8498 - val_rmse: 1.0973
Epoch 173/1300
16852/16852 [==============================] - 12s 697us/sample - loss: 1.9475 - rmse: 1.3088 - val_loss: 1.8191 - val_rmse: 1.0979
Epoch 174/1300
16852/16852 [==============================] - 12s 694us/sample - loss: 1.9485 - rmse: 1.3086 - val_loss: 1.8212 - val_rmse: 1.0966
Epoch 175/1300
16852/16852 [==============================] - 12s 710us/sample - loss: 1.9457 - rmse: 1.3076 - val_los

Epoch 225/1300
16852/16852 [==============================] - 12s 693us/sample - loss: 1.8979 - rmse: 1.2899 - val_loss: 1.7750 - val_rmse: 1.0727
Epoch 226/1300
16852/16852 [==============================] - 12s 694us/sample - loss: 1.8865 - rmse: 1.2866 - val_loss: 1.7624 - val_rmse: 1.0689
Epoch 227/1300
16852/16852 [==============================] - 12s 694us/sample - loss: 1.9077 - rmse: 1.2912 - val_loss: 1.7866 - val_rmse: 1.0806
Epoch 228/1300
16852/16852 [==============================] - 12s 690us/sample - loss: 1.8897 - rmse: 1.2873 - val_loss: 1.7519 - val_rmse: 1.0674
Epoch 229/1300
16852/16852 [==============================] - 12s 693us/sample - loss: 1.8957 - rmse: 1.2894 - val_loss: 1.7690 - val_rmse: 1.0726
Epoch 230/1300
16852/16852 [==============================] - 12s 705us/sample - loss: 1.8842 - rmse: 1.2851 - val_loss: 1.8530 - val_rmse: 1.0968
Epoch 231/1300
16852/16852 [==============================] - 12s 697us/sample - loss: 1.9022 - rmse: 1.2910 - val_los

...

Epoch 897/1300
16852/16852 [==============================] - 12s 708us/sample - loss: 1.5174 - rmse: 1.1398 - val_loss: 1.7011 - val_rmse: 0.9943
Epoch 898/1300
16852/16852 [==============================] - 12s 708us/sample - loss: 1.5373 - rmse: 1.1460 - val_loss: 1.5809 - val_rmse: 0.9668
Epoch 899/1300
16852/16852 [==============================] - 12s 708us/sample - loss: 1.5251 - rmse: 1.1413 - val_loss: 1.7011 - val_rmse: 0.9991
Epoch 900/1300
16852/16852 [==============================] - 12s 711us/sample - loss: 1.5174 - rmse: 1.1393 - val_loss: 1.6617 - val_rmse: 0.9836
Epoch 901/1300
16852/16852 [==============================] - 12s 709us/sample - loss: 1.5453 - rmse: 1.1487 - val_loss: 1.6115 - val_rmse: 0.9750
Epoch 902/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 1.5198 - rmse: 1.1409 - val_loss: 1.6894 - val_rmse: 0.9965
Epoch 903/1300
16852/16852 [==============================] - 12s 702us/sample - loss: 1.5230 - rmse: 1.1415 - val_los

Epoch 953/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 1.5076 - rmse: 1.1353 - val_loss: 1.6280 - val_rmse: 0.9723
Epoch 954/1300
16852/16852 [==============================] - 12s 703us/sample - loss: 1.5094 - rmse: 1.1350 - val_loss: 1.5980 - val_rmse: 0.9785
Epoch 955/1300
16852/16852 [==============================] - 12s 696us/sample - loss: 1.4906 - rmse: 1.1293 - val_loss: 1.5984 - val_rmse: 0.9640
Epoch 956/1300
16852/16852 [==============================] - 12s 697us/sample - loss: 1.4963 - rmse: 1.1302 - val_loss: 1.6897 - val_rmse: 1.0001
Epoch 957/1300
16852/16852 [==============================] - 12s 698us/sample - loss: 1.4792 - rmse: 1.1256 - val_loss: 1.5988 - val_rmse: 0.9661
Epoch 958/1300
16852/16852 [==============================] - 12s 697us/sample - loss: 1.4937 - rmse: 1.1334 - val_loss: 1.6021 - val_rmse: 0.9669
Epoch 959/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 1.5166 - rmse: 1.1377 - val_los

16852/16852 [==============================] - 12s 709us/sample - loss: 1.4735 - rmse: 1.1225 - val_loss: 1.6154 - val_rmse: 0.9710
Epoch 1009/1300
16852/16852 [==============================] - 12s 708us/sample - loss: 1.5051 - rmse: 1.1315 - val_loss: 1.6345 - val_rmse: 0.9701
Epoch 1010/1300
16852/16852 [==============================] - 12s 703us/sample - loss: 1.4875 - rmse: 1.1263 - val_loss: 1.6365 - val_rmse: 0.9681
Epoch 1011/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 1.4645 - rmse: 1.1191 - val_loss: 1.7436 - val_rmse: 1.0178
Epoch 1012/1300
16852/16852 [==============================] - 12s 709us/sample - loss: 1.4783 - rmse: 1.1233 - val_loss: 1.6007 - val_rmse: 0.9602
Epoch 1013/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 1.4819 - rmse: 1.1247 - val_loss: 1.6434 - val_rmse: 0.9753
Epoch 1014/1300
16852/16852 [==============================] - 12s 709us/sample - loss: 1.4950 - rmse: 1.1293 - val_loss: 1.6382

16852/16852 [==============================] - 12s 694us/sample - loss: 1.4703 - rmse: 1.1195 - val_loss: 1.5494 - val_rmse: 0.9466
Epoch 1064/1300
16852/16852 [==============================] - 12s 700us/sample - loss: 1.4868 - rmse: 1.1244 - val_loss: 1.5593 - val_rmse: 0.9607
Epoch 1065/1300
16852/16852 [==============================] - 12s 695us/sample - loss: 1.4544 - rmse: 1.1142 - val_loss: 1.6451 - val_rmse: 0.9742
Epoch 1066/1300
16852/16852 [==============================] - 12s 701us/sample - loss: 1.4670 - rmse: 1.1186 - val_loss: 1.6654 - val_rmse: 0.9830
Epoch 1067/1300
16852/16852 [==============================] - 12s 709us/sample - loss: 1.4605 - rmse: 1.1160 - val_loss: 1.6068 - val_rmse: 0.9618
Epoch 1068/1300
16852/16852 [==============================] - 12s 712us/sample - loss: 1.4810 - rmse: 1.1217 - val_loss: 1.5935 - val_rmse: 0.9571
Epoch 1069/1300
16852/16852 [==============================] - 12s 706us/sample - loss: 1.4719 - rmse: 1.1202 - val_loss: 1.6301

KeyboardInterrupt: 

In [24]:
lookid_dir = '../input/IdLookupTable.csv'
lookid_data = pd.read_csv(lookid_dir)
test_data = pd.read_csv(test_file)

x_test = []
for i in range(0,len(test_data)):
    img = test_data['Image'][i].split(' ')
    x_test.append(img)
    
x_test = np.array(x_test,dtype = 'float')
x_test = x_test/255.0
x_test = x_test.reshape(-1,96,96,1)    

y_test = model.predict(x_test)
y_test = np.clip(y_test,0,96)

lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pred_list = list(y_test)

rowid = list(lookid_data['RowId'])

feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
    
    
submit_data = []
for x,y in zip(imageID,feature):
    submit_data.append(pred_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(submit_data,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('../output/w207_temp_submission.csv',index = False)   


In [ ]:
if 1:
    sess = get_session()
    clear_session()
    sess.close()
 
    try:
        del model # this is from global space - change this as you need
    except:
        print("Model clear Failed")
    print(gc.collect())    